<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
STATUS MulAddFusionPass::AddNewScaleNode(MetaGraphT *graph, const std::unique_ptr<CNodeT> &mulNode, CNodeT *addNode,
                                         uint32_t addBiasIndex) {
  MS_ASSERT(graph != nullptr);
  MS_ASSERT(mulNode != nullptr);
  MS_ASSERT(addNode != nullptr);
  // replace mulNode as scale
  mulNode->primitive->value.type = schema::PrimitiveType_Scale;
  std::unique_ptr<ScaleT> scaleParam(new ScaleT());
  if (scaleParam == nullptr) {
    MS_LOG(ERROR) << "new transposeParam failed";
    return RET_ERROR;
  }
  // NHWC
  int shape_size = graph->allTensors.at(addBiasIndex)->dims.size();
  scaleParam->axis = 0 - shape_size;
  mulNode->inputIndex.push_back(addBiasIndex);
  auto activationType = addNode->primitive->value.AsAdd()->activationType;
  if (activationType == ActivationType_RELU || activationType == ActivationType_RELU6) {
    scaleParam->activationType = activationType;
    // delete addnode
    auto status = IsolateOneWayNode(graph, addNode);
    if (status != RET_OK) {
      MS_LOG(ERROR) << "IsolateOneWayNode failed";
      return status;
    }
  } else {
    std::unique_ptr<ActivationT> activationParam(new ActivationT());
    activationParam->type = addNode->primitive->value.AsAdd()->activationType;
    addNode->primitive->value.type = schema::PrimitiveType_Activation;
    addNode->primitive->value.value = activationParam.release();
    addNode->inputIndex.pop_back();
  }
  mulNode->primitive->value.value = scaleParam.release();
  return RET_OK;
}

In [ ]:
static const std::unordered_map<int, int> nc2NhAxisMap = {
    {0, 0}, {1, -1}, {2, 1}, {3, 2}
};

std::unordered_map<int, int> GetNc2NhAxisMap() { return nc2NhAxisMap; };
